<a href="https://colab.research.google.com/github/naveen333nav/LabWorkRepo/blob/master/cifar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
%tensorflow_version 1.x
import tensorflow as tf
import os
from keras.layers.core import Dense,Activation,Dropout
from keras.layers import Conv2D,MaxPool2D,Flatten
from keras.models import Sequential
from keras.layers import BatchNormalization
from keras.optimizers import SGD
from keras.utils import np_utils
from keras.datasets import cifar10
from matplotlib import pyplot
import h5py
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from keras.models import load_model


def load_dataset():
    (X_train,Y_train),(X_test,Y_test) = cifar10.load_data()

    #onehot encoding y values 
    y_train = np_utils.to_categorical(Y_train)
    y_test = np_utils.to_categorical(Y_test)

    return (X_train,X_test),(y_train,y_test)

#scale pixels (preprocessing image data

def preparing_pixels(X_train,X_test):
  #converting values to float
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
  
  #Normalizing values in range [0,1] 
    X_train = X_train/255.0
    X_test = X_test/255.0

    return X_train,X_test

print(tf.__version__)


1.15.0


In [0]:
resolver = tf.contrib.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.contrib.distribute.initialize_tpu_system(resolver)
strategy = tf.contrib.distribute.TPUStrategy(resolver)

In [0]:
def define_model():
    #defing VGG3 Architecture 
    model = Sequential()
    model.add(Conv2D(32,(3,3),activation='relu',input_shape=(32,32,3),padding='same',kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    model.add(Conv2D(32,(3,3),activation='relu',padding='same',kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    model.add(MaxPool2D((2,2)))
    model.add(Dropout(0.2))
    model.add(Conv2D(64,(3,3),activation='relu',padding='same',kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    model.add(Conv2D(64,(3,3),activation='relu',padding='same',kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(2,2))
    model.add(Dropout(0.3))
    model.add(Conv2D(128,(3,3),activation='relu',padding='same',kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    model.add(Conv2D(128,(3,3),activation='relu',padding='same',kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(2,2))
    model.add(Dropout(0.4))
    model.add(Flatten())
    model.add(Dense(128,activation='relu',kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))
    model.add(Dense(10,activation='softmax'))

 

    #compile model
    model.compile(optimizer=SGD(lr=0.001,momentum=0.9),loss='categorical_crossentropy',metrics=['accuracy'])

    
    return model
model = define_model()

In [0]:
def summarizing_model(history):
    #plot loss
    pyplot.subplot(211)
    pyplot.title('Cross entropy loss')
    pyplot.plot(history.history['loss'],color='red',label = 'train_loss')
    pyplot.plot(history.history['val_loss'],color='blue',label='test_loss')
    
    #plot accuracy
    pyplot.subplot(212)
    pyplot.title('Classification accuracy')
    pyplot.plot(history.history['acc'],color='red',label='train_accuracy')
    pyplot.plot(history.history['val_accuracy'],color= 'blue',label='test_loss')

    pyplot.close()

In [0]:

def run_test_harness():
    #loading dataset
    (X_train,X_test),(y_train,y_test) = load_dataset()



    #preparing dataset
    X_train , X_test = preparing_pixels(X_train,X_test)

    #defining model
    model = define_model()
    
    datagen = ImageDataGenerator(width_shift_range=0.1,height_shift_range=0.1,horizontal_flip=True)
    train_iterator = datagen.flow(X_train,y_train,batch_size=64)
    
    #fit the model 
    steps = int(X_train.shape[0]/64)
    history = model.fit_generator(train_iterator,steps_per_epoch=steps,epochs=150,validation_data=(X_test,y_test),verbose=1)
    
    model.save("final_model.h5")
    # #evaluating model
    # loss,acc = model.evaluate(X_test,y_test,verbose=1)
    # print('>%.5f' %(acc * 100.0))

    #learning curves 
    summarizing_model(history)

run_test_harness()

In [0]:
model  = load_model('final_model.h5')
# model.save_weights('final_weights.h5')
model.predict_classes(X_)


In [73]:
def load_image(filename):
    img = load_img(filename,target_size=(32,32))
    
    img = img_to_array(img)

    img = img.reshape(-1,32,32,3)
    
    img = img.astype('float32')
    img = img/255.0
    return img
  
def run_example():
    
    img = load_image("horse.jpg")
    model = load_model('final_model.h5')
    result = model.predict_classes(img)
    print(result[0])

run_example()  #outputs 4




7


In [72]:
from google.colab import files
from IPython.display import Image
uploaded = files.upload() 


Saving horse.jpg to horse.jpg


In [50]:



# (X_train,y_train),(X_test,y_test) = cifar10.load_data()
# test_ = X_test[2]
# test_=test_.reshape(1,32,32,3)
# test_=test_.astype('float32')

# test_ = test_/255.0
# result = model.predict_classes(test_)
# print(result[0],y_test[2])







8 [8]
